# Team member:
- Fulin ZHANG
- Ruicong WANG
- Yuekai FENG

In [45]:
import matplotlib.pyplot as plt

# 3. Importing & loading data

## Structure of network

In [46]:
markets = ['c1','c2','c3','c4','c5','c6','c7','c8'] # Country 1-8
plants = ['c1','c2','c3','c4','c5','c6'] # Country 1-6

## Capacity

In [47]:
manufacturing_capacity = {
    'c1' : 1725,
    'c2' : 800,
    'c3' : 5286,
    'c4' : 483,
    'c5' : 1050,
    'c6' : 2334,
}

## Fixed operating cost

In [48]:
prod_fixed_costs = {
    'c1' : 1380000,
    'c2' : 1050000,
    'c3' : 3171600,
    'c4' : 618240,
    'c5' : 1785000,
    'c6' : 1633800
}

## Demands

In [49]:
demand_markets = {
    'c1' : 1120,
    'c2' : 425,
    'c3' : 3225,
    'c4' : 292,
    'c5' : 684,
    'c6' : 2065,
    'c7' : 247,
    'c8' : 180
}

## Landed Cost

In [50]:
shipping_variable_costs = {
    #        c1    c2    c3    c4    c5    c6    c7    c8
    'c1' : [4840, 4900, 5484, 5501, 5182, 5050, 5152, 5172],
    'c2' : [5250, 5180, 5838, 5872, 5415, 5350, 5509, 5519],
    'c3' : [5030, 5508, 4880, 5626, 5236, 5080, 5030, 5314],
    'c4' : [5020, 5504, 5594, 4940, 5198, 5140, 5020, 5297],
    'c5' : [5191, 5708, 5788, 5830, 5071, 5311, 5211, 5425],
    'c6' : [5423, 5655, 5775, 5776, 5436, 5050, 5170, 5493]
}

## Lead Time Matrix

In [51]:
lead_time_matrix = {
    #      c1  c2  c3  c4  c5  c6  c7  c8
    'c1': [ 7, 18, 24, 14, 31, 31, 14, 39],
    'c2': [12,  7, 15, 12, 25, 20, 12, 25],
    'c3': [16, 15,  7, 16, 27, 25, 16, 29],
    'c4': [18, 20, 27,  7, 31, 21, 21, 23],
    'c5': [31, 38, 38, 38,  7, 38, 38, 24],
    'c6': [36, 40, 33, 32, 44,  7, 40, 49]
    }

# Max lead time allowed is 30 days.

# 4. Data analysis

In [52]:
total_demand = 0
for key,value in demand_markets.items():
    total_demand += value

print("total demande in the market is ", total_demand)

total demande in the market is  8238


In [53]:
total_capacity = 0
for key,value in manufacturing_capacity.items():
    total_capacity += value

print("total capacity of the plants is ", total_capacity)

total capacity of the plants is  11678


In [54]:
# Initialize the total cost variable
total_cost_before_optimization = 0


# Add fixed costs
for value in prod_fixed_costs.values():
    total_cost_before_optimization += value

# Current supply chain
current_supply_chain = {
    'c1': {'c1': 1120},
    'c2': {'c2': 425},
    'c3': {'c3': 3225, 'c7': 247, 'c8': 180},
    'c4': {'c4': 292},
    'c5': {'c5': 684},
    'c6': {'c6': 2065}
}

# Add shipping costs
for country, markets_dict in current_supply_chain.items():
    for market, amount in markets_dict.items():
        market_index = markets.index(market)
        total_cost_before_optimization += amount * shipping_variable_costs[country][market_index]

print("Total cost before optimization is:", total_cost_before_optimization)


Total cost before optimization is: 50537164


# 5. Optimization -- Build the model

Decidions to make：
1. Maximize utilization of production capacity
2. Minimize costs

restrictions:

1. reserve 700t of idle production capacity
2. Delivery time must not be longer than 30 days
3. Market demand coverage

In [55]:
from gurobipy import Model, quicksum, GRB

In [56]:
supplyChainModel = Model('SupplyChainOptimization')

# add a a binary variable for each plant to indicate if it should be open or not
openVar = supplyChainModel.addVars(plants, vtype=GRB.BINARY, name='open')

# add a continuous variable for the production of each plant to each market
# for example, the variable productionVar['c1','c1'] is the production of plant c1 to market c1
allocationVar = supplyChainModel.addVars(markets, markets, vtype=GRB.CONTINUOUS, name='allocation')

Restricted license - for non-production use only - expires 2025-11-24


In [57]:
# Constraint 1:
# The sum of production of each plant should be less than the manufacturing capacity of the plant

supplyChainModel.addConstrs(
    (quicksum(allocationVar[plant_id, market_id] for market_id in markets) 
     <= 
     manufacturing_capacity[plant_id]
     for plant_id in plants),
    "CapacityConstraint"
)

{'c1': <gurobi.Constr *Awaiting Model Update*>,
 'c2': <gurobi.Constr *Awaiting Model Update*>,
 'c3': <gurobi.Constr *Awaiting Model Update*>,
 'c4': <gurobi.Constr *Awaiting Model Update*>,
 'c5': <gurobi.Constr *Awaiting Model Update*>,
 'c6': <gurobi.Constr *Awaiting Model Update*>}

In [58]:
# Constraint 2:
# The sum of production of all plants should cover the sum of demand of each market

supplyChainModel.addConstrs(
    (quicksum(allocationVar[plant_id, market_id] for plant_id in plants) 
     >= 
     demand_markets[market_id]
     for market_id in markets),
    "DemandConstraint"
)

{'c1': <gurobi.Constr *Awaiting Model Update*>,
 'c2': <gurobi.Constr *Awaiting Model Update*>,
 'c3': <gurobi.Constr *Awaiting Model Update*>,
 'c4': <gurobi.Constr *Awaiting Model Update*>,
 'c5': <gurobi.Constr *Awaiting Model Update*>,
 'c6': <gurobi.Constr *Awaiting Model Update*>,
 'c7': <gurobi.Constr *Awaiting Model Update*>,
 'c8': <gurobi.Constr *Awaiting Model Update*>}

In [59]:
# Constraint 3:
# If the lead time for a plant to a market is greater than 30 days, then the production should be 0

supplyChainModel.addConstrs(
    (allocationVar[plant_id, market_id] == 0
     for plant_id in plants for market_id in markets if lead_time_matrix[plant_id][markets.index(market_id)] > 30),
    "LeadTimeConstraint"
)

{('c1', 'c5'): <gurobi.Constr *Awaiting Model Update*>,
 ('c1', 'c6'): <gurobi.Constr *Awaiting Model Update*>,
 ('c1', 'c8'): <gurobi.Constr *Awaiting Model Update*>,
 ('c4', 'c5'): <gurobi.Constr *Awaiting Model Update*>,
 ('c5', 'c1'): <gurobi.Constr *Awaiting Model Update*>,
 ('c5', 'c2'): <gurobi.Constr *Awaiting Model Update*>,
 ('c5', 'c3'): <gurobi.Constr *Awaiting Model Update*>,
 ('c5', 'c4'): <gurobi.Constr *Awaiting Model Update*>,
 ('c5', 'c6'): <gurobi.Constr *Awaiting Model Update*>,
 ('c5', 'c7'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c1'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c2'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c3'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c4'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c5'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c7'): <gurobi.Constr *Awaiting Model Update*>,
 ('c6', 'c8'): <gurobi.Constr *Awaiting Model Update*>}

In [60]:
# Constraint 4:
# Should have at least 700 units of spare capacity after meeting the demand

supplyChainModel.addConstr(
    quicksum(manufacturing_capacity[plant_id] for plant_id in plants) - 
    quicksum(allocationVar[plant_id, market_id] for plant_id in plants for market_id in markets)
    >= 700,
    "TotalSpareCapacityConstraint"
)

<gurobi.Constr *Awaiting Model Update*>

In [61]:
# Constraint 5:
# If a plant is not open, then the production should be 0

for plant_id in plants:
    supplyChainModel.addConstr(
        quicksum(allocationVar[plant_id, market_id] for market_id in markets) <= manufacturing_capacity[plant_id] * openVar[plant_id],
        name=f"CapacityUtilization_{plant_id}"
    )

In [62]:
# shipping cost = shipping_variable_costs * allocationVar
total_shipping_cost = quicksum(
    shipping_variable_costs[plant_id][markets.index(market_id)] * allocationVar[plant_id, market_id]
    for plant_id in plants for market_id in markets
)

# fixed cost = openVar * prod_fixed_costs
total_fixed_cost_with_opening = quicksum(openVar[plant_id] * prod_fixed_costs[plant_id] for plant_id in plants)

total_cost = total_shipping_cost + total_fixed_cost_with_opening

# 设置模型的目标
supplyChainModel.setObjective(total_cost, GRB.MINIMIZE)


In [63]:
supplyChainModel.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 38 rows, 70 columns and 215 nonzeros
Model fingerprint: 0x04c50618
Variable types: 64 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [5e+03, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 1e+04]
Presolve removed 24 rows and 35 columns
Presolve time: 0.00s
Presolved: 14 rows, 35 columns, 92 nonzeros
Variable types: 30 continuous, 5 integer (5 binary)
Found heuristic solution: objective 4.936499e+07

Root relaxation: objective 4.658442e+07, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.6584e+07    0    3 4.9365e+07 4.6584e+07  5.63%  

In [64]:
print("total cost before optimization is ", total_cost_before_optimization)
print("total cost after optimization is ",supplyChainModel.ObjVal)

print("\nSaving is ", total_cost_before_optimization - supplyChainModel.ObjVal)

total cost before optimization is  50537164
total cost after optimization is  47255596.0

Saving is  3281568.0


In [65]:
total_shipping_cost.getValue()

41070196.0

In [66]:
total_capacity = 0

for plant_id in plants:
    for market_id in markets:
        if allocationVar[plant_id, market_id].X != 0:
            print('plant:',plant_id,'market :',market_id,' ', allocationVar[plant_id, market_id].X)
            total_capacity += allocationVar[plant_id, market_id].X

print('total capacity:', total_capacity)

plant: c1 market : c1   1120.0
plant: c1 market : c2   425.0
plant: c1 market : c4   180.0
plant: c3 market : c3   3225.0
plant: c3 market : c4   112.0
plant: c3 market : c5   684.0
plant: c3 market : c7   247.0
plant: c3 market : c8   180.0
plant: c6 market : c6   2065.0
total capacity: 8238.0


In [67]:
# plants opening status
for plant_id in plants:
    print('plant:',plant_id,' ',"Open" if openVar[plant_id].X == 1 else "Close")

plant: c1   Open
plant: c2   Close
plant: c3   Open
plant: c4   Close
plant: c5   Close
plant: c6   Open
